<a href="https://colab.research.google.com/github/Chenjie-UTS/UTS_ML2019-ID12769194/blob/master/A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import seaborn as sns
import matplotlib.pyplot as plt
import csv


In [0]:
from pathlib import Path
DATA_URL = 'https://drive.google.com/open?id=1f2g7LEB1LN7M4MZT_rA4wv06ep6js-ne'
DATA_FILENAME = "listings_dec18.csv.zip"

data_dir = Path("./data/")
# Create dir structure if not existing
data_dir.mkdir(parents=True, exist_ok=True) 
filepath = data_dir / DATA_FILENAME

data_source = "Google Drive"
#@markdown ---

if not filepath.exists():
    if data_source == 'Existing':
       raise ValueError("File doesn't exist, plese choose a source to get the file")
    elif data_source == "Google Drive":
        # Install the PyDrive wrapper & import libraries.
        # This only needs to be done once per notebook.
        !pip install -U -q PyDrive
        from pydrive.auth import GoogleAuth
        from pydrive.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials

        # Authenticate and create the PyDrive client.
        # This only needs to be done once per notebook.
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        # Download a file based on its file ID.
        #
        # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
        file_id = '1f2g7LEB1LN7M4MZT_rA4wv06ep6js-ne'
        downloaded = drive.CreateFile({'id': file_id})
        downloaded.GetContentFile(str(filepath))
    elif data_source == "NOAA/ERSL":
        import urllib.request
        urllib.request.urlretrieve(DATA_URL, str(filepath))

In [7]:
import zipfile
zf = zipfile.ZipFile(filepath)
zf.extractall('./data/')
df = pd.read_csv(zf.open('listings_dec18.csv'), encoding = "ISO-8859-1")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
# Import listings, load as pandas dataframe
listings_data = pd.read_csv('./data/listings_dec18.csv')
#Create dataframe listings with relevant columns （ drop 'neigbourhood group' column as NULL）
listings = listings_data[['id','listing_url','scrape_id','last_scraped','name','summary','space','description','experiences_offered','neighborhood_overview','notes','transit','access','interaction','house_rules','thumbnail_url','medium_url','picture_url','xl_picture_url','host_id']]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


idlisting ID

namename of the listing

host_idhost ID

host_namename of the host

neighbourhood_grouplocation

neighbourhoodarea

latitudelatitude coordinates

longitudelongitude coordinates

room_typelisting space type

priceprice in dollars

minimum_nightsamount of nights minimum

number_of_reviewsnumber of reviews

last_reviewlatest review

reviews_per_monthnumber of reviews per month

calculated_host_listings_countamount of listing per host

availability_365number of days when listing is available for booking

In [9]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36662 entries, 0 to 36661
Data columns (total 20 columns):
id                       36662 non-null int64
listing_url              36662 non-null object
scrape_id                36662 non-null int64
last_scraped             36662 non-null object
name                     36650 non-null object
summary                  35642 non-null object
space                    25052 non-null object
description              36116 non-null object
experiences_offered      36662 non-null object
neighborhood_overview    21617 non-null object
notes                    14907 non-null object
transit                  22096 non-null object
access                   21668 non-null object
interaction              20263 non-null object
house_rules              21429 non-null object
thumbnail_url            0 non-null float64
medium_url               0 non-null float64
picture_url              36662 non-null object
xl_picture_url           0 non-null float64
host_id  

In [10]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id
0,12351,https://www.airbnb.com/rooms/12351,20181207034750,2018-12-07,Sydney City & Harbour at the door,Come stay with Vinh & Stuart (Awarded as one o...,"We're pretty relaxed hosts, and we fully appre...",Come stay with Vinh & Stuart (Awarded as one o...,none,"Pyrmont is an inner-city village of Sydney, on...",We've a few reasons for the 6.00pm arrival tim...,Our home is centrally located and an easy walk...,We look forward to welcoming you just as we wo...,As much or as little as you like. We live here...,We look forward to welcoming you to stay you j...,NaN,NaN,https://a0.muscache.com/im/pictures/763ad5c8-c...,NaN,17061
1,14250,https://www.airbnb.com/rooms/14250,20181207034750,2018-12-07,Manly Harbour House,"Beautifully renovated, spacious and quiet, our...",Our home is a thirty minute walk along the sea...,"Beautifully renovated, spacious and quiet, our...",none,Balgowlah Heights is one of the most prestigio...,NaN,Balgowlah - Manly bus # 131 or #132 (Bus stop...,Guests have access to whole house except locke...,NaN,Standard Terms and Conditions of Temporary Hol...,NaN,NaN,https://a0.muscache.com/im/pictures/56935671/f...,NaN,55948
2,15253,https://www.airbnb.com/rooms/15253,20181207034750,2018-12-07,Stunning Penthouse Apartment In Heart Of The City,Penthouse living in a great central location: ...,"A charming two-level, two-bedroom, two-bathroo...",Penthouse living in a great central location: ...,none,The location is really central and there is nu...,The apartment is on the top floor with 4 fligh...,If walking is not your thing I am well service...,Guests have own key and access to use faciliti...,"You would be staying with me and my very cute,...",I am fairly easygoing and will try to accomoda...,NaN,NaN,https://a0.muscache.com/im/pictures/3643414/2e...,NaN,59850
3,20865,https://www.airbnb.com/rooms/20865,20181207034750,2018-12-07,3 BED HOUSE + 1 BED STUDIO Balmain,Hi! We are a married professional couple with ...,HOUSE : _________ * DUCTED AIR CONDITIONING IN...,Hi! We are a married professional couple with ...,none,BALMAIN is an older inner city village / subur...,During your stay guests may use the landline f...,Balmain is well serviced by buses (from Darlin...,Whole house (including studio/flat & courtyard...,We will always be available should you require...,PLEASE ENJOY YOURSELVES WITHOUT MAKING TOO MUC...,NaN,NaN,https://a0.muscache.com/im/pictures/58785124/c...,NaN,64282
4,26174,https://www.airbnb.com/rooms/26174,20181207034750,2018-12-07,"COZY PRIVATE ROOM, GREAT LOCATION!",NaN,"Double bed in decent sized bedroom, in two bed...","Double bed in decent sized bedroom, in two bed...",none,NaN,NaN,NaN,NaN,NaN,Need to be quiet after 10.30pm No Smoking Clea...,NaN,NaN,https://a0.muscache.com/im/pictures/7ce9714f-8...,NaN,110561


In [0]:
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import array
import datetime
#from sklearn.tree import DecisionTreeClassifier

In [0]:
label_encoder = LabelEncoder()
copylistings = listings.copy();

copylistings['price'] = copylistings['price'].astype(float)

#copylistings['lr_year'] = pd.DatetimeIndex(copylistings['last_review']).year
#copylistings['lr_month'] = pd.DatetimeIndex(copylistings['last_review']).month
#copylistings['lr_day'] = pd.DatetimeIndex(copylistings['last_review']).month

copylistings['rt_encode']= label_encoder.fit_transform(listings['room_type'])
copylistings['nbh_encode']= label_encoder.fit_transform(listings['neighbourhood'])

copylistings.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,rt_encode,nbh_encode
0,12351,Sydney City & Harbour at the door,17061,Stuart,Sydney,-33.865153,151.191896,Private room,100.0,2,493,2018-11-21,4.83,2,187,1,32
1,14250,Manly Harbour House,55948,Heidi,Manly,-33.800929,151.261722,Entire home/apt,471.0,5,1,2016-01-02,0.03,2,321,0,20
2,15253,Stunning Penthouse Apartment In Heart Of The City,59850,Morag,Sydney,-33.880455,151.216541,Private room,109.0,2,300,2018-11-16,3.63,2,316,1,32
3,20865,3 BED HOUSE + 1 BED STUDIO Balmain,64282,Fiona,Leichhardt,-33.859072,151.172753,Entire home/apt,450.0,7,15,2018-01-07,0.18,1,69,0,18
4,26174,"COZY PRIVATE ROOM, GREAT LOCATION!",110561,Amanda,Woollahra,-33.889087,151.259404,Private room,62.0,1,42,2018-11-23,0.45,1,140,1,37


In [0]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(copylistings, test_size=0.2)

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(algorithm='brute')


cols = ['number_of_reviews','rt_encode','nbh_encode','availability_365','minimum_nights','calculated_host_listings_count']
#normalized_copylistings[] = preprocessing.normalize(copylistings)
#normalizer = preprocessing.Normalizer().fit(cols)  # fit does nothing
#normalizer
#Normalizer(copy=True, norm='l1')

knn.fit(train_data[cols], train_data['price'])
predict_data = knn.predict(test_data[cols])

test_data['predict_data']= knn.predict(test_data[cols])

test_data.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,rt_encode,nbh_encode,predict_data
20286,21489422,Large Sydney Harbour Side Family House,131520066,Sebastian,North Sydney,-33.841302,151.227294,Entire home/apt,999.0,14,0,NaN,NaN,1,0,0,23,83.8
28677,26460105,Stylish north shore apartment with panoramic v...,16683254,Damien,North Sydney,-33.826870,151.226420,Private room,100.0,1,2,2018-07-24,0.41,1,78,1,23,246.2
12033,15128165,Bright North Ryde House - perfect for short stay,69497152,Tina,Ryde,-33.798506,151.135972,Private room,48.0,5,16,2017-12-08,0.59,1,84,1,29,127.0
28797,26585474,Bright Ocean View Room Bondi,121542820,Emma,Waverley,-33.888675,151.262700,Private room,46.0,1,0,NaN,NaN,1,0,1,35,75.8
14210,16416700,North Bondi Private Room Queen Bed,107586059,D'Arcy,Waverley,-33.881577,151.270343,Private room,37.0,7,4,2017-07-29,0.20,1,0,1,35,81.2


In [0]:
from sklearn.metrics import mean_squared_error
test_data['squared_error'] = (test_data['predict_data'] - test_data['price'])**(2)
mse = test_data['squared_error'].mean()
rmse = mse ** (1/2)
rmse


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


289.36709452899845

In [0]:
print("RMSE for the {} column: {}".format('number_of_reviews',rmse))
print("RMSE for the {} column: {}".format('rt_encode',rmse))
print("RMSE for the {} column: {}".format('nbh_encode',rmse))
print("RMSE for the {} column: {}".format('availability_365',rmse))
print("RMSE for the {} column: {}".format('minimum_nights',rmse))
print("RMSE for the {} column: {}".format('calculated_host_listings_count',rmse))



RMSE for the number_of_reviews column: 289.36709452899845
RMSE for the rt_encode column: 289.36709452899845
RMSE for the nbh_encode column: 289.36709452899845
RMSE for the availability_365 column: 289.36709452899845
RMSE for the minimum_nights column: 289.36709452899845
RMSE for the calculated_host_listings_count column: 289.36709452899845
